In [1]:
import pandas as pd
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

2023-06-03 18:28:37.164960: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-06-03 18:28:37.320980: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [3]:
# Define the dataset
course = pd.read_csv('course_clean.csv')
course.set_index('judul', inplace=True)

In [4]:
# Create TF-IDF vectorizer
tf = TfidfVectorizer(analyzer='word', ngram_range=(1, 3), min_df=0, stop_words='english')
tfidf_matrix = tf.fit_transform(course['summary'])
cos_sim = cosine_similarity(tfidf_matrix, tfidf_matrix)

In [5]:
# Train the neural network model
def train_model(cos_sim, tfidf_matrix):
    input_data = cos_sim.reshape(len(dataset1), -1)
    output_data = tfidf_matrix.toarray()

    # Define the neural network model
    model = Sequential()
    model.add(Dense(32, activation='relu', input_shape=(input_data.shape[1],)))
    model.add(Dropout(0.2))
    model.add(Dense(64, activation='relu'))
    model.add(Dense(output_data.shape[1], activation='linear'))

    # Compile the model
    model.compile(loss='mse', optimizer='adam')

    # Train the model
    model.fit(input_data, output_data, epochs=10, batch_size=32)

    return model

In [6]:
# Train the model
model = train_model(cos_sim, tfidf_matrix)

2023-06-03 18:29:40.267393: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:982] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-06-03 18:29:40.301529: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:982] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-06-03 18:29:40.301568: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:982] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-06-03 18:29:40.305873: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:982] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-06-03 18:29:40.305948: I tensorflow/compile

Epoch 1/10


2023-06-03 18:29:49.504854: I tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:637] TensorFloat-32 will be used for the matrix multiplication. This will only be logged once.
2023-06-03 18:29:49.599454: I tensorflow/compiler/xla/service/service.cc:169] XLA service 0x7f4420045520 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
2023-06-03 18:29:49.599489: I tensorflow/compiler/xla/service/service.cc:177]   StreamExecutor device (0): NVIDIA GeForce RTX 3050 Laptop GPU, Compute Capability 8.6
2023-06-03 18:29:49.683072: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:269] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
2023-06-03 18:29:49.995402: I tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:424] Loaded cuDNN version 8600
2023-06-03 18:29:50.671656: I ./tensorflow/compiler/jit/device_compiler.h:180] Compiled cluster using XLA!  This line is logged at most once for the lifeti

1/1 [==============================] - 9s 9s/step - loss: 0.0010
Epoch 2/10
1/1 [==============================] - 0s 17ms/step - loss: 9.3537e-04
Epoch 3/10
1/1 [==============================] - 0s 9ms/step - loss: 8.3649e-04
Epoch 4/10
1/1 [==============================] - 0s 8ms/step - loss: 7.6723e-04
Epoch 5/10
1/1 [==============================] - 0s 8ms/step - loss: 7.2027e-04
Epoch 6/10
1/1 [==============================] - 0s 10ms/step - loss: 6.7195e-04
Epoch 7/10
1/1 [==============================] - 0s 7ms/step - loss: 6.6632e-04
Epoch 8/10
1/1 [==============================] - 0s 8ms/step - loss: 5.9142e-04
Epoch 9/10
1/1 [==============================] - 0s 8ms/step - loss: 5.5815e-04
Epoch 10/10
1/1 [==============================] - 0s 6ms/step - loss: 5.6621e-04


In [7]:
# Generate recommendations
def recommendations(name, model, dataset, cos_sim):
    recommended_course = []

    # Get the index of the input course title
    idx = dataset.index.get_loc(name)

    # Reshape the cosine similarity matrix for the input data
    input_data = cos_sim.reshape(len(dataset), -1)

    # Predict the TF-IDF matrix for the input course
    predicted_tfidf = model.predict(input_data[idx].reshape(1, -1)).reshape(-1,)

    # Calculate the cosine similarity between the predicted and actual TF-IDF matrices
    score_series = pd.Series(cosine_similarity(predicted_tfidf.reshape(1, -1), tfidf_matrix).flatten())

    # Get the top 10 indexes with the highest cosine similarity scores
    top_10_indexes = list(score_series.sort_values(ascending=False).iloc[1:5].index)

    for i in top_10_indexes:
        recommended_course.append(dataset.index[i])

    return recommended_course

In [9]:
# Generate recommendations for a specific course
title = 'Moringa Academy'
recommended_course = recommendations(title, model, dataset1, cos_sim)
print(recommended_course)

1/1 [==============================] - 0s 14ms/step
['Indonesia Sustainable Social Forestry Education Program (IS-FREE)', 'Program Startup', 'Indonesia Sustainability Coral Reef University Network (ISCORE)', 'Ecotourism']
